In [2]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import math
import time

In [3]:
def threshold(matrix, threshold):
    matrix[matrix > threshold]  = 255
    matrix[matrix <= threshold]  = 0
    return matrix

def negative(matrix):
    matrix =  255 - matrix 
    return matrix

def log_transform(matrix, c, normalize=False):
    # Substitui valores zero por um valor pequeno para evitar logaritmo de zero
    matrix = np.where(matrix == 0, 1e-10, matrix)
    matrix = c * np.log(1 + matrix)
    
    if normalize:
        # Calcula o valor máximo do logaritmo da imagem para normalização
        max_log = c * np.log(1 + np.max(matrix))
        matrix = (matrix / max_log) * 255  # Normaliza para o intervalo [0, 255]
    
    matrix = np.clip(matrix, 0, 255).astype(np.uint8)
    return matrix

def exponential(matrix, c, gamma):
    matrix = matrix.astype(np.float32)
    matrix = c * np.power(matrix, gamma)
    matrix = np.clip(matrix, 0, 255).astype(np.uint8)
    return matrix

def equalize_histogram(frame):
    # Equalizando o canal vermelho manualmente
    hist, bins = np.histogram(frame.flatten(), 256, [0, 256])
    cdf = hist.cumsum()
    cdf_normalized = cdf * hist.max() / cdf.max()
    cdf_masked = np.ma.masked_equal(cdf, 0)
    cdf_masked = (cdf_masked - cdf_masked.min()) * 255 / (cdf_masked.max() - cdf_masked.min())
    cdf_final = np.ma.filled(cdf_masked, 0).astype('uint8')
    equalized = cdf_final[frame]

    # Combinando os canais de cor de volta em uma imagem
    return equalized

# Questão 1

In [4]:
frame = cv.Mat
frame_ = cv.Mat
paused = False
cap = cv.VideoCapture('../src/video.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
times = []  
while cap.isOpened():
    if not paused:
        start_time = time.time()
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        frame = cv.resize(frame, (640, 480))
        
        B, G, R = cv.split(frame)
        R = threshold(R, 128)
        G = threshold(G, 128)
        B = threshold(B, 128)
        frame_ = cv.merge((B, G, R))

        end_time = time.time()

        processing_time = end_time - start_time
        times.append(processing_time)

        # Calculate the average processing time
        average_time = sum(times) / len(times)

        # Put the average processing time on the frame
        text = f"Avg time: {average_time:.4f} sec"
        cv.putText(frame, text, (10, 450), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
    
    cv.imshow('frame', frame)
    cv.imshow('frame_', frame_)

    key = cv.waitKey(30)
    if key == 27:  # Esc key to exit
        break
    elif key == ord('p'):  # Pause
        paused = True
    elif key == ord('o'):  # Continue
        paused = False

cap.release()
cv.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/kaio/.local/lib/python3.10/site-packages/cv2/qt/plugins"


 A imagem original, que continha uma rica variedade de tons e detalhes, é convertida em uma representação altamente simplificada. As transições suaves entre cores e as sombras são eliminadas, deixando áreas claramente definidas onde os valores de cor foram mantidos ou descartados.

# Questão 2

In [8]:
frame = cv.Mat
frame_ = cv.Mat
paused = False
cap = cv.VideoCapture('../src/video.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
times = []  
while cap.isOpened():
    if not paused:
        start_time = time.time()
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        frame = cv.resize(frame, (640, 480))
        
        B, G, R = cv.split(frame)
        R = negative(R)
        G = negative(G)
        B = negative(B)
        frame_ = cv.merge((B, G, R))

        end_time = time.time()

        processing_time = end_time - start_time
        times.append(processing_time)

        # Calculate the average processing time
        average_time = sum(times) / len(times)

        # Put the average processing time on the frame
        text = f"Avg time: {average_time:.4f} sec"
        cv.putText(frame, text, (10, 450), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
    
    cv.imshow('frame', frame)
    cv.imshow('frame_', frame_)

    key = cv.waitKey(30)
    if key == 27:  # Esc key to exit
        break
    elif key == ord('p'):  # Pause
        paused = True
    elif key == ord('o'):  # Continue
        paused = False

cap.release()
cv.destroyAllWindows()

As cores vibrantes e familiares da imagem original são substituídas por versões opostas. As áreas claras da imagem se tornam escuras e vice-versa, enquanto as cores quentes se transformam em suas equivalentes frias.

# Questão 3

In [21]:
frame = cv.Mat
frame_ = cv.Mat
paused = False
cap = cv.VideoCapture('../src/video.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
times = []
c_values = [10, 30, 50]
while cap.isOpened():
    if not paused:
        start_time = time.time()
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        frame = cv.resize(frame, (640, 480))

        B, G, R = cv.split(frame)
        transformed_frames = []
        for c in c_values:
            R_transformed = log_transform(R, c, normalize=False)
            G_transformed = log_transform(G, c, normalize=False)
            B_transformed = log_transform(B, c, normalize=False)
            transformed_frame = cv.merge((B_transformed, G_transformed, R_transformed))
            transformed_frames.append(transformed_frame)

        R_normalized = log_transform(R, c_values[-1], normalize=True)
        G_normalized = log_transform(G, c_values[-1], normalize=True)
        B_normalized = log_transform(B, c_values[-1], normalize=True)
        normalized_frame = cv.merge((B_normalized, G_normalized, R_normalized))

        end_time = time.time()

        processing_time = end_time - start_time
        times.append(processing_time)

        average_time = sum(times) / len(times)

        for i, transformed_frame in enumerate(transformed_frames):
            cv.imshow(f'frame c={c_values[i]}', transformed_frame)

        cv.imshow('Normalized frame', normalized_frame)
        cv.imshow('frame', frame)

        text = f"Avg time: {average_time:.4f} sec"
        cv.putText(frame, text, (10, 450), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
    
    key = cv.waitKey(30)
    if key == 27:  # Esc key to exit
        break
    elif key == ord('p'):  # Pause
        paused = True
    elif key == ord('o'):  # Continue
        paused = False

cap.release()
cv.destroyAllWindows()


O efeito visual varia conforme o valor de cc. Com valores baixos de cc, a imagem transformada se torna mais escura, com realce dos detalhes nas áreas de menor intensidade, enquanto as áreas mais claras são suavizadas. Com valores maiores de cc, a imagem aparece mais brilhante, mas com menor contraste nas áreas de alta intensidade.

# Questão 4

In [16]:
frame = cv.Mat
frame_ = cv.Mat
paused = False
cap = cv.VideoCapture('../src/video.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
times = []  
while cap.isOpened():
    if not paused:
        start_time = time.time()
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        frame = cv.resize(frame, (640, 480))
        
        B, G, R = cv.split(frame)
        R = exponential(R, 0.3, 1.3)
        G = exponential(G, 0.3, 1.3)
        B = exponential(B, 0.3, 1.3)
        frame_ = cv.merge((B, G, R))

        end_time = time.time()

        processing_time = end_time - start_time
        times.append(processing_time)

        # Calculate the average processing time
        average_time = sum(times) / len(times)

        # Put the average processing time on the frame
        text = f"Avg time: {average_time:.4f} sec"
        cv.putText(frame, text, (10, 450), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
    
    cv.imshow('frame', frame)
    cv.imshow('frame_', frame_)

    key = cv.waitKey(30)
    if key == 27:  # Esc key to exit
        break
    elif key == ord('p'):  # Pause
        paused = True
    elif key == ord('o'):  # Continue
        paused = False

cap.release()
cv.destroyAllWindows()

O impacto visual da transformação depende dos valores de cc e γγ. Com γ<1γ<1, a transformação realça os valores de intensidade mais baixos, tornando as áreas escuras mais brilhantes, enquanto as áreas claras são pouco afetadas, resultando em uma imagem mais clara e com mais detalhes visíveis nas sombras. Com γ>1γ>1, o efeito é oposto: a transformação comprime os valores de intensidade, tornando as áreas escuras ainda mais escuras e intensificando o brilho das áreas claras, o que pode levar à perda de detalhes nas sombras.

# Questão 5

In [5]:
frame = cv.Mat
frame_ = cv.Mat
_frame_ = cv.Mat
paused = False
cap = cv.VideoCapture('../src/video.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
times = []  
while cap.isOpened():
    if not paused:
        start_time = time.time()
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        frame = cv.resize(frame, (640, 480))
        
        B, G, R = cv.split(frame)
        R_ = cv.equalizeHist(R)
        R = equalize_histogram(R)
        frame_ = cv.merge((B, G, R))

        end_time = time.time()

        processing_time = end_time - start_time
        times.append(processing_time)

        # Calculate the average processing time
        average_time = sum(times) / len(times)

        # Put the average processing time on the frame
        text = f"Avg time: {average_time:.4f} sec"
        cv.putText(frame, text, (10, 450), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
    
    cv.imshow('frame', frame)
    cv.imshow('frame_', frame_)

    key = cv.waitKey(30)
    if key == 27:  # Esc key to exit
        break
    elif key == ord('p'):  # Pause
        paused = True
    elif key == ord('o'):  # Continue
        paused = False

cap.release()
cv.destroyAllWindows()

Após a aplicação da equalização de histograma exclusivamente no canal vermelho, a imagem resultante ficou com uma tonalidade avermelhada. Isso ocorre porque a redistribuição dos valores de intensidade no canal vermelho faz com que ele se torne mais dominante em relação aos outros canais (verde e azul).

# Questão 6

In [6]:
frame = cv.Mat
frame_ = cv.Mat
paused = False
cap = cv.VideoCapture('../src/video.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
times = []  
while cap.isOpened():
    if not paused:
        start_time = time.time()
        ret, frame = cap.read()
        if not ret:
            print("Can’t receive frame (stream end?). Exiting ...")
            break
        frame = cv.resize(frame, (640, 480))
        frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        frame_ = threshold(frame, np.mean(frame))

        kernel = np.ones((9, 9), np.uint8)

        frame_ = cv.erode(frame_, kernel, iterations=1)
        frame_ = cv.dilate(frame_, kernel, iterations=1)

        end_time = time.time()

        processing_time = end_time - start_time
        times.append(processing_time)

        # Calculate the average processing time
        average_time = sum(times) / len(times)

        # Put the average processing time on the frame
        text = f"Avg time: {average_time:.4f} sec"
        cv.putText(frame, text, (10, 450), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
    
    cv.imshow('frame', frame)
    cv.imshow('frame_', frame_)

    key = cv.waitKey(30)
    if key == 27:  # Esc key to exit
        break
    elif key == ord('p'):  # Pause
        paused = True
    elif key == ord('o'):  # Continue
        paused = False

cap.release()
cv.destroyAllWindows()

O vídeo transformado apresenta uma redução significativa nos detalhes, com contornos e formas simplificados. A aplicação de erosão e dilatação destaca ainda mais as bordas dos objetos e pode criar efeitos de "desgaste" ou "inchaço" nas formas. O resultado é um vídeo que enfatiza as formas e estruturas principais, perdendo as gradações suaves e detalhes presentes no vídeo original.